In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import fnmatch
import os
from matplotlib import colors
import matplotlib.pyplot as plt

2021-12-13 00:47:23.275076: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-13 00:47:23.275111: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%run data_collection.ipynb

In [3]:
model = keras.models.load_model('model_weights/model_1_accuracy_100.model')

2021-12-13 00:47:32.444289: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-13 00:47:32.444332: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (coco): /proc/driver/nvidia/version does not exist
2021-12-13 00:47:32.444853: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
initialize(4,0.3)
final_grid = encode_grid(grid,[])
final_grid[0,0] = 4

In [5]:
model.predict(np.asarray([final_grid.tolist()]))

array([[6.0505552e-21, 9.8645246e-01, 1.3547548e-02, 6.2458529e-23]],
      dtype=float32)

In [6]:
initialize(10,0.3)

In [7]:
def patch_of_grid(grid,current_position):
    global maze_size
    mat =  grid.copy()
    row = current_position[0]
    col = current_position[1]
    
    col_left = col- 1
    col_right = col + 2
    
    row_high = row - 1
    row_low = row + 2
    
    # print("initial cols:",col_left,col_right,"rows: ",row_high,row_low)
    if(col_left<0): # workng
        difference = 0-col_left
        col_left = 0 
        col_right += difference
        
    if(col_right>maze_size-1):
        difference = col_right - maze_size + 1
        col_left -= difference
        col_right = maze_size - 1
        
    if(row_low>maze_size-1):
        difference = row_low - maze_size + 1
        print(difference)
        row_low = maze_size - 1 
        row_high -= difference
    if(row_high<0):
        difference = 0-row_high
        row_high = 0
        row_low += difference
    
    return mat[row_high:row_low+1,col_left:col_right+1]

In [8]:
mat = patch_of_grid(grid,(0,5))

In [27]:
grid

matrix([[0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
        [1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0]])

In [10]:
mat

matrix([[1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 0]])

In [34]:
def run_ml_1_agent_2(model,grid):
    global maze_size
    
    
    visited = []
    goal_reached = False
    goal = (maze_size-1,maze_size-1)
    counter = 0
    agent_grid = np.asmatrix(np.zeros(maze_size*maze_size, dtype=int).reshape(maze_size,maze_size))
    
    current_position = (0,0)
    while not goal_reached and counter!=10:
        print("initial current position: ",current_position)
        
        if current_position not in visited:
            visited.append(current_position)
            
        # check if current_position ia the goal
        if(current_position==goal):
            print("goal reached")
            return True
         
        #else get direction
        patch = patch_of_grid(agent_grid,current_position)
        print("predicting for: \n",patch)
        patch = encode_grid(patch,visited)
        direction = model.predict(np.asarray([patch.tolist()]))[0]
#         print(direction)
        direction = np.argmax(direction)
        print(direction)
        
        #update coordinates according to direction
        row = current_position[0]
        col = current_position[1]
        if(direction == 0 and row-1>=0): #Up
            row -= 1
        elif(direction == 1 and row+1<maze_size): #down
            row += 1
        elif(direction == 2 and col-1>=0): #left
            col -=1
        elif(col+1<maze_size): #right
            col +=1
            
        # check if new current position. if yes, update agent_grid
        # and don't update current position
        if(grid[row,col]==1):
#             print(row,col)
            agent_grid[row,col]=1
#             print("hit block, agent_grid now:",agent_grid)
            counter += 1
            continue
        #else update current position
        current_position = (row,col)
        
        print("final current_position: ",current_position)
        counter += 1
    return False
    

In [35]:
run_ml_1_agent_2(model,grid)

initial current position:  (0, 0)
predicting for: 
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
final current_position:  (0, 1)
initial current position:  (0, 1)
predicting for: 
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
3
initial current position:  (0, 1)
predicting for: 
 [[0 0 1 0]
 [0 0 0 0]
 [0

False